In [15]:
# stardist / tensorflow env variables setup
import os
os.environ["OMP_NUM_THREADS"] = "4"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from pathlib import Path

import napari
import numpy as np
from napari.utils.notebook_display import nbscreenshot
from tqdm import tqdm
from rich.pretty import pprint

from stardist.models import StarDist2D

from ultrack import track, to_tracks_layer, tracks_to_zarr
from ultrack.imgproc import normalize
from ultrack.utils import estimate_parameters_from_labels, labels_to_contours
from ultrack.utils.array import array_apply
from ultrack.config import MainConfig

# 1. stardist track

In [16]:
#import
dataset_path = Path("images")

viewer = napari.Viewer()
viewer.window.resize(1800, 1000)
viewer.open(sorted(dataset_path.glob("*.tif")), stack=True)

image = viewer.layers[0].data

In [17]:
model = StarDist2D.from_pretrained("2D_versatile_fluo")
stardist_labels = np.zeros_like(image, dtype=np.int32)

def predict(frame: np.ndarray, model: StarDist2D) -> np.ndarray:
    """Normalizes and computes stardist prediction."""
    frame = normalize(frame, gamma=1.0)
    labels, _ = model.predict_instances_big(
        frame, "YX", block_size=1024, min_overlap=192, show_progress=False,
    )
    return labels

array_apply(
    image,
    out_array=stardist_labels,
    func=predict,
    model=model,
)

viewer.add_labels(stardist_labels, name="stardist")

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


Applying predict ...:   0%|          | 0/44 [00:00<?, ?it/s]

effective: block_size=(1024, 1024), min_overlap=(192, 192), context=(96, 96)


Applying predict ...:   2%|▏         | 1/44 [00:02<01:54,  2.66s/it]

effective: block_size=(1024, 1024), min_overlap=(192, 192), context=(96, 96)


...

effective: block_size=(1024, 1024), min_overlap=(192, 192), context=(96, 96)


Applying predict ...:  98%|█████████▊| 43/44 [01:13<00:01,  1.77s/it]

effective: block_size=(1024, 1024), min_overlap=(192, 192), context=(96, 96)


Applying predict ...: 100%|██████████| 44/44 [01:15<00:00,  1.72s/it]


<Labels layer 'stardist' at 0x3713b7fd0>